<a href="https://colab.research.google.com/github/Yomex139/NLP/blob/Exercise_1/Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Importing Libraries

In [ ]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_hub as hub
from tensorflow.keras import layers

!wget https://github.com/Yomex139/Food-Vision/raw/main/Helper_Function.py

--2023-09-19 17:57:52--  https://github.com/Yomex139/Food-Vision/raw/main/Helper_Function.py
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Yomex139/Food-Vision/main/Helper_Function.py [following]
--2023-09-19 17:57:52--  https://raw.githubusercontent.com/Yomex139/Food-Vision/main/Helper_Function.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13819 (13K) [text/plain]
Saving to: ‘Helper_Function.py’

Helper_Function.py  100%[===================>]  13.50K  --.-KB/s    in 0s      

2023-09-19 17:57:52 (77.9 MB/s) - ‘Helper_Function.py’ saved [13819/13819]



In [ ]:
from Helper_Function import pred_and_plot,unzip_data,create_tensorboard_callback,text_metrics

## Downloading and unziping dataset

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-09-19 17:57:55--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 142.250.101.207, 2607:f8b0:4023:c0d::cf, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2023-09-19 17:57:55 (125 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
unzip_data('nlp_getting_started.zip')

## read in dataset using pandas csv

In [ ]:
# read in our train data using pandas csv
train_df = pd.read_csv('train.csv')
# reshuffled train data
train_df_shuffled = train_df.sample(frac=1)
train_df_shuffled.head()

,id,keyword,location,text,target
4685,6661,landslide,New York City,Inbounds/ Out of Bounds:\n\nWhile many picked ...,1
6217,8871,smoke,NaN,Be Trynna smoke TJ out but he a hoe,0
3743,5320,fire,NaN,My avi And header combo is fire,0
5880,8399,sandstorm,United States,I liked a @YouTube video http://t.co/xR3xJJ8gJ...,0
7564,10813,wrecked,NaN,@__ohhmyjoshh @stevenrulles he not gonna be th...,0


In [ ]:
from sklearn.model_selection import train_test_split
train_sentence,val_sentence,train_label,val_label = train_test_split(train_df_shuffled['text'].to_list(),
                                                                     train_df_shuffled['target'].to_list(),
                                                                     test_size=.1,
                                                                     random_state=5)

In [ ]:
len(train_sentence),len(train_label),len(val_sentence),len(val_label)

(6851, 6851, 762, 762)

## visiualize train sentence with the target label

In [ ]:
from IPython.utils.tempdir import TemporaryWorkingDirectory
import random
rand = random.randint(0,5)
for row in train_df_shuffled[['text','target']][rand:rand+13].itertuples():
  _,text,target = row
  print(f'\nTarget:{target}' , 'disatster' if target>0 else 'not disater\n')
  print(f'\nText:\n{text}\n')
  print('\n--------\n')



Target:0 not disater


Text:
Wolforth with a two-out single up the middle. Fourth hit of the night for Altamonte Springs.


--------


Target:0 not disater


Text:
Survived Spanish!! @ Sweet Ritual https://t.co/inIqzfyioi


--------


Target:1 disatster

Text:
CANADA BC DROUGHT: Okanagan region issued Level 4 rating - Okanagan River (Columbia trib) fishing suspended to Sep 30 http://t.co/r4yZHxk7lw


--------


Target:1 disatster

Text:
Malik Saadthe best police officer produced by KPdied in a suicide bombing on 27 Jan 2007he was more than a brother to me. #KPPolice.


--------


Target:0 not disater


Text:
I'm an emotional wreck watching emmerdale


--------


Target:0 not disater


Text:
Queens Gambit went well until Anakin blew up the droid control ship. Oh well still fun! Can't Stop is next! #WBC2015


--------


Target:1 disatster

Text:
Myanmar: MSF assisting thousands as worst flooding in decades hits parts of Myanmar http://t.co/PiJG5w2L2u


--------


Target:1 disatster

Tex

## Tokinization and embedding

In [ ]:
max_vocab_length=15000
output_length=20

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding
text_vectorize = TextVectorization(max_tokens=max_vocab_length,
                                   output_sequence_length=output_length)
text_vectorize.adapt(train_sentence)

text_embed = Embedding(input_dim=max_vocab_length,
                       output_dim=256,
                       input_length=[])

In [ ]:
my_name = text_vectorize(['my name is daniel'])

text_embed(my_name)

<tf.Tensor: shape=(1, 20, 256), dtype=float32, numpy=
array([[[-0.04790428, -0.01480597, -0.02503701, ..., -0.04305091,
         -0.01877791,  0.01246711],
        [ 0.00956459, -0.0474996 , -0.00503575, ..., -0.04149836,
         -0.03006986, -0.04149749],
        [-0.02624282, -0.03780996, -0.00189593, ..., -0.014465  ,
          0.03984395,  0.04198425],
        ...,
        [-0.03714907,  0.03090069,  0.03492118, ...,  0.01105996,
         -0.02784652,  0.01992197],
        [-0.03714907,  0.03090069,  0.03492118, ...,  0.01105996,
         -0.02784652,  0.01992197],
        [-0.03714907,  0.03090069,  0.03492118, ...,  0.01105996,
         -0.02784652,  0.01992197]]], dtype=float32)>

## Modelling
text data has been converted to numbers next is creating models

In [ ]:
#  base model(model 01)
from tensorflow.keras import layers
base_model = tf.keras.Sequential([
                                  layers.Input([],dtype=tf.string),
                                  text_vectorize,
                                  text_embed,
                                  layers.GlobalAveragePooling1D(),
                                  layers.Dense(64,activation='relu'),
                                  layers.Dense(1,activation='sigmoid')
])

#Compile the model
base_model.compile(loss='binary_crossentropy',
                   optimizer='adam',
                   metrics=['accuracy'])

#Fit the model
base_model.fit(x=train_sentence,
               y=train_label,
               epochs=10,
               validation_data=(val_sentence,val_label),
               callbacks=create_tensorboard_callback(dir_name='NLP_excercise',
                                                     experiment_name='base model'))

Saving TensorBoard log files to: NLP_excercise/base model/20230919-175800
Epoch 1/10
215/215 [==============================] - 23s 81ms/step - loss: 0.5442 - accuracy: 0.7218 - val_loss: 0.4356 - val_accuracy: 0.8136
Epoch 2/10
215/215 [==============================] - 4s 19ms/step - loss: 0.3108 - accuracy: 0.8743 - val_loss: 0.4546 - val_accuracy: 0.8018
Epoch 3/10
215/215 [==============================] - 3s 14ms/step - loss: 0.1845 - accuracy: 0.9307 - val_loss: 0.5559 - val_accuracy: 0.7835
Epoch 4/10
215/215 [==============================] - 2s 11ms/step - loss: 0.1143 - accuracy: 0.9587 - val_loss: 0.6679 - val_accuracy: 0.7677
Epoch 5/10
215/215 [==============================] - 2s 11ms/step - loss: 0.0765 - accuracy: 0.9717 - val_loss: 0.7745 - val_accuracy: 0.7664
Epoch 6/10
215/215 [==============================] - 1s 6ms/step - loss: 0.0557 - accuracy: 0.9781 - val_loss: 0.8631 - val_accuracy: 0.7572
Epoch 7/10
215/215 [==============================] - 2s 9ms/step - 

In [ ]:
base_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVe  (None, 20)                0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 20, 256)           3840000   
                                                                 
 global_average_pooling1d (  (None, 256)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 64)                16448     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 3856513 (14.71 MB)
Trainable params: 38565

In [ ]:
# Evaluate on validation sentence
base_model.evaluate(val_sentence,val_label)
# predict on val sentence
base_pred = base_model.predict(val_sentence)
base_pred = tf.round(tf.squeeze(base_pred))
base_pred

24/24 [==============================] - 0s 4ms/step


<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0.,
       1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1.,
       0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
       1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0.,
       1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
       0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0.,
       1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1.,
       0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

# Building model 2 using LSTM

In [ ]:
# model 01(LSTM)
model_01 = tf.keras.Sequential([
                                layers.Input([],dtype='string'),
                                text_vectorize,
                                text_embed,
                                layers.LSTM(units=128,return_sequences=True),
                                layers.MaxPool1D(pool_size=8),
                                layers.GlobalAveragePooling1D(),
                                layers.Dense(1,activation='sigmoid')
])
# Compile the model
model_01.compile(loss='binary_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])
# Fit the model
model_01.fit(x=train_sentence,
             y=train_label,
             epochs=10,
             validation_data=(val_sentence,val_label),
             callbacks=create_tensorboard_callback(dir_name='NLP_excercise',
                                                   experiment_name='LSTM model'))

Saving TensorBoard log files to: NLP_excercise/LSTM model/20230919-175907
Epoch 1/10
215/215 [==============================] - 18s 63ms/step - loss: 0.1465 - accuracy: 0.9526 - val_loss: 0.7581 - val_accuracy: 0.7546
Epoch 2/10
215/215 [==============================] - 5s 24ms/step - loss: 0.0578 - accuracy: 0.9766 - val_loss: 0.9546 - val_accuracy: 0.7612
Epoch 3/10
215/215 [==============================] - 3s 12ms/step - loss: 0.0365 - accuracy: 0.9848 - val_loss: 1.0367 - val_accuracy: 0.7585
Epoch 4/10
215/215 [==============================] - 3s 15ms/step - loss: 0.0343 - accuracy: 0.9870 - val_loss: 1.0158 - val_accuracy: 0.7559
Epoch 5/10
215/215 [==============================] - 2s 11ms/step - loss: 0.0282 - accuracy: 0.9879 - val_loss: 0.9611 - val_accuracy: 0.7572
Epoch 6/10
215/215 [==============================] - 3s 14ms/step - loss: 0.0231 - accuracy: 0.9886 - val_loss: 1.2632 - val_accuracy: 0.7533
Epoch 7/10
215/215 [==============================] - 2s 10ms/step 

In [ ]:
model_01.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVe  (None, 20)                0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 20, 256)           3840000   
                                                                 
 lstm (LSTM)                 (None, 20, 128)           197120    
                                                                 
 max_pooling1d (MaxPooling1  (None, 2, 128)            0         
 D)                                                              
                                                                 
 global_average_pooling1d_1  (None, 128)               0         
  (GlobalAveragePooling1D)                                       
                                                      

In [ ]:
# prediction with LSTM model
LSTM_pred = model_01.evaluate(val_sentence,val_label)
LSTM_pred = tf.round(tf.squeeze(LSTM_pred))
LSTM_pred

24/24 [==============================] - 0s 6ms/step - loss: 1.2275 - accuracy: 0.7546


<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 1.], dtype=float32)>

# Building model 3(SKlearn Multinominal with tf-idf(term frequency-times inverse document frequency)

> Indented block



In [ ]:
, TfidfTransformer
# Building model_02
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

model_02 = Pipeline([
                     ('tf-idf',TfidfVectorizer()),
                     ('Multinominal',MultinomialNB())
])

# Fit the model
model_02.fit(train_sentence,train_label)

Pipeline(steps=[('tf-idf', TfidfVectorizer()),
                ('Multinominal', MultinomialNB())])

In [ ]:
model_02.score(val_sentence,val_label)

0.7939632545931758

# Building mode_03 using tranfer learning from tensorflow hub

In [ ]:
# Create 10% of the dataset
import tensorflow_hub as hub
train_10 = int(.1*len(train_df_shuffled))
train_10_sentence = train_df_shuffled['text'][:train_10]
train_10_label = train_df_shuffled['target'][:train_10]

In [ ]:
Encoder = hub.KerasLayer(
                          'https://tfhub.dev/google/universal-sentence-encoder/4',
                          input_shape=[],
                          dtype=tf.string,
                          output_shape=[20],
                          trainable=False
)

In [ ]:
model_03 = tf.keras.Sequential()
model_03.add(Encoder)
model_03.add(layers.Dense(128,activation='relu'))
model_03.add(layers.Dense(64,activation='relu'))
model_03.add(layers.Dense(1,activation='sigmoid'))

# Compile the model
model_03.compile(loss='binary_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])

model_03.fit(x=train_10_sentence,
             y=train_10_label,
             epochs=10,
             validation_data=(val_sentence,val_label),
             callbacks=create_tensorboard_callback(dir_name='NLP_exercise',
                                                   experiment_name='Hub 10 percent_trainable_false'))

Saving TensorBoard log files to: NLP_exercise/Hub 10 percent_trainable_false/20230919-180017
Epoch 1/10
24/24 [==============================] - 6s 43ms/step - loss: 0.6602 - accuracy: 0.6071 - val_loss: 0.6062 - val_accuracy: 0.7060
Epoch 2/10
24/24 [==============================] - 1s 23ms/step - loss: 0.5159 - accuracy: 0.8147 - val_loss: 0.4828 - val_accuracy: 0.7940
Epoch 3/10
24/24 [==============================] - 1s 27ms/step - loss: 0.3967 - accuracy: 0.8357 - val_loss: 0.4496 - val_accuracy: 0.8123
Epoch 4/10
24/24 [==============================] - 1s 27ms/step - loss: 0.3275 - accuracy: 0.8686 - val_loss: 0.4335 - val_accuracy: 0.8189
Epoch 5/10
24/24 [==============================] - 1s 23ms/step - loss: 0.2786 - accuracy: 0.8909 - val_loss: 0.4626 - val_accuracy: 0.8031
Epoch 6/10
24/24 [==============================] - 1s 23ms/step - loss: 0.2308 - accuracy: 0.9133 - val_loss: 0.4765 - val_accuracy: 0.8097
Epoch 7/10
24/24 [==============================] - 1s 26ms/s

In [ ]:
model_03.evaluate(val_sentence,val_label)

24/24 [==============================] - 0s 10ms/step - loss: 0.5501 - accuracy: 0.8123


[0.5500509738922119, 0.8123359680175781]

# Setting trainable to true and train on 10% of the dataset

In [ ]:
encoder = hub.KerasLayer(
                          'https://tfhub.dev/google/universal-sentence-encoder/4',
                          input_shape=[],
                          output_shape=output_length,
                          dtype='string',
                          trainable=True
)

In [ ]:
model_04 = tf.keras.Sequential()
model_04.add(Encoder)
model_04.add(layers.Dense(64,activation='relu'))
model_04.add(layers.Dense(1,activation='sigmoid'))

# compile the model
model_04.compile(loss='binary_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])

# fit the model
model_04.fit(x=train_10_sentence,
             y=train_10_label,
             epochs=10,
             validation_data=(val_sentence,val_label),
             callbacks=create_tensorboard_callback(dir_name='NLP_exercise',
                                                   experiment_name='hub model trainable set to True'))

Saving TensorBoard log files to: NLP_exercise/hub model trainable set to True/20230919-180036
Epoch 1/10
24/24 [==============================] - 3s 44ms/step - loss: 0.6652 - accuracy: 0.7057 - val_loss: 0.6328 - val_accuracy: 0.7651
Epoch 2/10
24/24 [==============================] - 1s 27ms/step - loss: 0.5860 - accuracy: 0.8160 - val_loss: 0.5596 - val_accuracy: 0.7913
Epoch 3/10
24/24 [==============================] - 1s 26ms/step - loss: 0.5053 - accuracy: 0.8279 - val_loss: 0.4981 - val_accuracy: 0.7979
Epoch 4/10
24/24 [==============================] - 1s 26ms/step - loss: 0.4452 - accuracy: 0.8397 - val_loss: 0.4665 - val_accuracy: 0.8058
Epoch 5/10
24/24 [==============================] - 1s 40ms/step - loss: 0.4073 - accuracy: 0.8449 - val_loss: 0.4521 - val_accuracy: 0.8097
Epoch 6/10
24/24 [==============================] - 1s 27ms/step - loss: 0.3819 - accuracy: 0.8436 - val_loss: 0.4438 - val_accuracy: 0.8110
Epoch 7/10
24/24 [==============================] - 1s 26ms/

In [ ]:
model_04.evaluate(val_sentence,val_label)

24/24 [==============================] - 0s 17ms/step - loss: 0.4347 - accuracy: 0.8058


[0.4346855878829956, 0.8057742714881897]

In [ ]:
model_04.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 512)               256797824 
                                                                 
 dense_6 (Dense)             (None, 64)                32832     
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


# Training on full dataset

In [ ]:
model_05 = tf.keras.models.clone_model(model_04)

In [ ]:
# compile the model
model_04.compile(loss='binary_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])

# fit the model
model_04.fit(x=train_sentence,
             y=train_label,
             epochs=10,
             validation_data=(val_sentence,val_label),
             callbacks=create_tensorboard_callback(dir_name='NLP_exercise',
                                                   experiment_name='hub model trainable set to True on full dataset'))

Saving TensorBoard log files to: NLP_exercise/hub model trainable set to True on full dataset/20230919-180050
Epoch 1/10
215/215 [==============================] - 4s 14ms/step - loss: 0.4269 - accuracy: 0.8078 - val_loss: 0.4265 - val_accuracy: 0.8150
Epoch 2/10
215/215 [==============================] - 3s 15ms/step - loss: 0.4017 - accuracy: 0.8243 - val_loss: 0.4231 - val_accuracy: 0.8018
Epoch 3/10
215/215 [==============================] - 3s 13ms/step - loss: 0.3909 - accuracy: 0.8301 - val_loss: 0.4236 - val_accuracy: 0.7953
Epoch 4/10
215/215 [==============================] - 3s 12ms/step - loss: 0.3823 - accuracy: 0.8368 - val_loss: 0.4246 - val_accuracy: 0.8058
Epoch 5/10
215/215 [==============================] - 4s 17ms/step - loss: 0.3738 - accuracy: 0.8389 - val_loss: 0.4253 - val_accuracy: 0.8123
Epoch 6/10
215/215 [==============================] - 3s 12ms/step - loss: 0.3683 - accuracy: 0.8397 - val_loss: 0.4246 - val_accuracy: 0.8018
Epoch 7/10
215/215 [============

In [ ]:
model_04.evaluate(val_sentence,val_label)

24/24 [==============================] - 0s 10ms/step - loss: 0.4310 - accuracy: 0.8136


[0.4310232996940613, 0.8136482834815979]

In [ ]:
pd.read_csv('/content/sample_submission.csv')

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0


In [ ]:
test_data = pd.read_csv('test.csv')
test_sentence = test_data['text'].to_numpy()
test_sentence

array(['Just happened a terrible car crash',
       'Heard about #earthquake is different cities, stay safe everyone.',
       'there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all',
       ..., 'Green Line derailment in Chicago http://t.co/UtbXLcBIuY',
       'MEG issues Hazardous Weather Outlook (HWO) http://t.co/3X6RBQJHn3',
       '#CityofCalgary has activated its Municipal Emergency Plan. #yycstorm'],
      dtype=object)

In [ ]:
predictions = model_04.predict(test_sentence)
predictions = tf.round(tf.squeeze(predictions))
predictions

102/102 [==============================] - 2s 14ms/step


<tf.Tensor: shape=(3263,), dtype=float32, numpy=array([0., 1., 1., ..., 1., 1., 1.], dtype=float32)>

In [ ]:
subm = pd.DataFrame({
                     'id':test_data['id'],
                     'target':predictions
})

In [ ]:
subm['target'] = subm['target'].apply(np.int32)


In [ ]:
subm

,id,target
0,0,0
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [ ]:
subm.to_csv('text_data_submission')

In [ ]:
pd.read_csv('/content/text_data_submission')

,Unnamed: 0,id,target
0,0,0,0
1,1,2,1
2,2,3,1
3,3,9,1
4,4,11,1
...,...,...,...
3258,3258,10861,1
3259,3259,10865,1
3260,3260,10868,1
3261,3261,10874,1
